In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pymesh
#https://pymesh.readthedocs.io/en/latest/basic.html
# https://github.com/PyMesh/PyMesh


import meshplot
# for display of meshes
#https://skoch9.github.io/meshplot/tutorial/

import random

from multiprocessing import Pool #  Process pool
from multiprocessing import sharedctypes
import time
from functools import partial
#MULTITHREADING Adjustment
#Sample Multithreading Code 
# from https://jonasteuwen.github.io/numpy/python/multiprocessing/2017/01/07/multiprocessing-numpy-array.html
# https://thebinarynotes.com/python-multiprocessing/


# nicer plots
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

#from pathos.multiprocessing import ProcessingPool as Poolp

import quaternion
import sys
# shape models available here:
#https://www.naic.edu/~smarshal/1999kw4.html
#Squannit is secondary of Moshup which was 1999KW4


#### Problems:
our BYORP coeff seems consistent with the BYORP B.
But squannit is giving B  of order 1 or 5 or so.
This is 2 orders of mag too big.

We could have an error in the quaturnion

We could have an error in the face normals.

We could havve an error in the face areas.

### code for BYORP calculation


The surface thermal inertia is neglected, so that thermal radiation is re-emitted with no time lag, and the reflected and thermally radiated components are assumed Lambertian (isotropic) and so emitted with flux
parallel to the local surface normal. We ignore heat conduction. The surface is described with a closed
triangular mesh.


The radiation force from the $i$-th facet is
$$ {\bf F}_i  = - \frac{F_\odot}{c} {S_i} (\hat {\bf n}_i \cdot \hat {\bf s}_\odot) \hat {\bf n}_i $$
where  $S_i$ is the area of the $i$-th facet and $\hat {\bf n}_i$ is its surface normal.
Here $F_\odot$ is the solar radiation flux and $c$ is the speed of light.
The direction of the Sun is $\hat {\bf s}_\odot$.

The total Yarkovsky force is a sum over all the facets 
$${\bf F}_Y = \sum_{i: \hat {\bf n}_i \cdot \hat {\bf s}_\odot >0} {\bf F}_i $$
Only facets on the day side  or with $\hat {\bf n}_i \cdot \hat {\bf s}_\odot >0$ 
are included in the sum.

The torque affecting the binary orbit from a single facet is 
$$ {\boldsymbol \tau}_{i,B} = 
\begin{cases} 
- \frac{F_\odot}{c} {S_i} (\hat {\bf n}_i \cdot \hat {\bf s}_\odot) ( {\bf a}_B \times \hat {\bf n}_i)  
 & \mbox{if } \hat {\bf n}_i \cdot \hat {\bf s}_\odot >0  \\
 0 & \mbox{otherwise}
 \end{cases}
$$
where ${\bf a}_B$ is the secondary's radial vector from the binary center of mass.


The torque affecting the binary orbit is the sum of the torques from each facet and should be an average 
over the orbit around the Sun and 
over the binary orbit and spin of the secondary.
$$ {\boldsymbol \tau}_{BY} = \frac{1}{T} \int_0^T dt\   \sum_{i: \hat {\bf n}_i \cdot \hat {\bf s}_\odot >0} 
{\boldsymbol \tau}_{i,B} $$


If $\hat {\bf l}$ is the binary orbit normal then 
$$ {\boldsymbol \tau}_{BY} \cdot \hat {\bf l} $$ 
changes the binary's orbital angular momentum and causes binary orbit migration.


The torque affecting the spin (also known as YORP) instantaneously depends on 
the radii of each facit ${\bf r}_i$ from the asteroid center of mass 
$$ {\boldsymbol \tau}_{i,s}  = \begin{cases}
- \frac{F_\odot}{c} {S_i} (\hat {\bf n}_i \cdot \hat {\bf s}_\odot)  ({\bf r}_i \times \hat{\bf n}_i) 
 & \mbox{if } \hat {\bf n}_i \cdot \hat {\bf s}_\odot >0  \\
0 & \mbox{otherwise}
\end{cases}$$


$$ {\boldsymbol \tau}_Y = \frac{1}{T} \int_0^T dt \  \sum_{i: \hat {\bf n}_i \cdot \hat {\bf s}_\odot >0} {\boldsymbol \tau}_{i,s} $$
where the average is done over the orbit about the Sun and the spin of the asteroid.
If the spin axis is $\hat {\boldsymbol \omega}$ then 
$$ {\boldsymbol \tau}_Y \cdot \hat {\boldsymbol \omega}  $$ gives the body spin up or spin down rate.


In practice we average over the Sun's directions first and then average over spin (for YORP) or and spin and binary orbit direction (for BYORP) afterward.


<b> Units </b> 

For our calculation are $F_\odot/c = 1$.

For YORP $R=1$.
For BYORP $a_B = 1$ and $R=1$ (in the surface area).

Here $R$ is volume equivalent sphere radius.

To put in physical units: 

Multiply ${\boldsymbol \tau}_Y$ by $\frac{F_\odot R^3}{c}$.

Multiply ${\boldsymbol \tau}_{BY}$ by $\frac{F_\odot R^2 a_B}{c}$.

Alternatively we are computing:

${\boldsymbol \tau}_Y \times \frac{c}{F_\odot R^3} $ 

${\boldsymbol \tau}_{BY} \times \frac{c}{F_\odot R^2 a_B} $ 


To get the rate the spin changes for YORP

$\dot \omega = \frac{ {\boldsymbol \tau}_Y \cdot \hat {\bf s} }{C} $

where $C$ is the moment of inertia about the spin axis.

To order of magnitude what we are computing can be multiplied by 
$\frac{F_\odot R^3}{c MR^2} $ to estimate $\dot \omega$
and by $\frac{F_\odot R^3}{c MR^2 \omega} $
to estimate $\dot \epsilon$. 

To get the rate that obliquity changes for YORP

$\dot \epsilon = \frac{ {\boldsymbol \tau}_Y \cdot \hat {\boldsymbol \phi} }{C \omega} $

where unit vector $\hat {\boldsymbol \phi}$ is in the xy plane (ecliptic) and is perpendicular to the spin axis.

To get the semi-major axis drift rate for BYORP

$ \dot a_B = \frac{2 {\boldsymbol \tau}_{BY} \cdot \hat {\bf l}}{M n_Ba_B} $

where $M$ is the secondary mass, $n_B$ and $a_B$ are binary orbit mean motion and semi-major axis.

To order of magnitude to get the drift rate we multiply what we are getting by 
$\frac{F_\odot R^2 a_B}{c} \times \frac{1}{M n_B a_B}$.


Dimensionless numbers used by Steiberg+10 (eqns 19,48)

$f_{Y} \equiv \tau_{Y} \frac{3}{2} \frac{c}{\pi R^3 F_\odot}$

$f_{BY} \equiv \tau_{BY} \frac{3}{2} \frac{c}{\pi R^2 a_B F_\odot}$

Our computed values are  the same as theirs except for a factor of 3/2 
(but they have a 2/3 in their torque) and a factor of $\pi$.
We need to divide by $\pi$ to have values consistent with theirs.

<b> Assumptions:</b>

Circular orbit for binary.

Circuilar orbit for binary around Sun.

No shadows.

No conduction. Lambertian isotropic emission. No thermal lag.

We neglect distance of facet centroids from secondary center of mass when computing BYORP. 

Coordinate system:
binary orbit is kept in xy plane

Compare YORP on primary to BYORP on secondary.

$\frac{\tau_{Yp}}{\tau_{BY} }\sim \frac{R_p^2 }{R_s^2 } \frac{R_p }{a_B }\frac{f_Y}{ f_{BY}}$

For Didymos, this is about $8 f_Y/f_{BY}$.

In [2]:
random.seed(1)
print(random.uniform(-1,1))

-0.7312715117751976


In [3]:
# perturb a sphere (mesh, premade) and stretch it so that
# it becomes an ellipsoid.  
#    We can't directly edit vertices or faces
#    see this:  https://github.com/PyMesh/PyMesh/issues/156
#    the work around is to copy the entire mesh after modifying it
# arguments:
#   devrand,  Randomly add devrand to x,y,z positions of each vertex
#     a uniform ditns in [-1,1] is used
#   aratio1 and aratio2,  stretch or compress a sphere by aratio1 and aratio2 
# returns: a new mesh
# we assume that longest semi-major axis a is along x, 
#    medium semi-axis b is along y, semi-minor c axis is along z
# Volume should stay the same!
# this routine should work on any shaped body
def sphere_perturb(sphere,devrand,aratio1,aratio2):
    #devrand = 0.05  # how far to perturb each vertex
    nv = len(sphere.vertices)
    f = sphere.faces
    v = np.copy(sphere.vertices)
    # add perturbations to x,y,z to all vertices
    for i in range(nv):
        dx = devrand*random.uniform(-1,1)
        dy = devrand*random.uniform(-1,1)
        dz = devrand*random.uniform(-1,1)
        v[i,0] += dx
        v[i,1] += dy
        v[i,2] += dz
        
    
    # aratio1 = c/a  this gives c = aratio1*a
    # aratio2 = b/a  this gives b = aratio2*a
    # volume = 4/3 pi a*b*c for an ellipsoid
    # vol = 1*aratio1*aratio2
    # rad_cor = pow(vol,-1./3.)
    # v[:,2] *= aratio1*rad_cor # make oblate, adjusts z coords
    # v[:,1] *= aratio2*rad_cor # make elongated in xy plane , adjusts y coords
    #  v[:,0] *= rad_cor # adjusts x coords
    # volume should now stay the same
        
    sub_com(v) # subtract center of mass from vertex positions
    psphere = pymesh.form_mesh(v, f)
    psphere.add_attribute("face_area")
    psphere.add_attribute("face_normal")
    psphere.add_attribute("face_centroid")
    
    sbody = body_stretch(psphere,aratio1,aratio2)  # do the stretching 
    return sbody
                         

# stretch a mesh body by axis ratios 
# arguments:
#    body: mesh
#    aratio1: c/a
#    aratio2: b/a
# returns: a new mesh
# we assume that longest semi-major axis a is along x, 
#    medium semi-axis b is along y, semi-minor c axis is along z
# Volume should stay the same!
def body_stretch(body,aratio1,aratio2):
    nv = len(body.vertices)
    f = body.faces
    v = np.copy(body.vertices)
    
    # aratio1 = c/a  this gives c = aratio1*a
    # aratio2 = b/a  this gives b = aratio2*a
    # volume = 4/3 pi a*b*c for an ellipsoid
    vol = 1*aratio1*aratio2
    rad_cor = pow(vol,-1./3.)
    v[:,2] *= aratio1*rad_cor # make oblate, adjusts z coords
    v[:,1] *= aratio2*rad_cor # make elongated in xy plane , adjusts y coords
    v[:,0] *= rad_cor # adjusts x coords
    # volume should now stay the same
    
    sub_com(v) # subtract center of mass from vertex positions
    sbody = pymesh.form_mesh(v, f)
    sbody.add_attribute("face_area")
    sbody.add_attribute("face_normal")
    sbody.add_attribute("face_centroid")
    return sbody
    

# substract the center of mass from a list of vertices
def sub_com(v):
    nv = len(v)
    xsum = np.sum(v[:,0])
    ysum = np.sum(v[:,1])
    zsum = np.sum(v[:,2])
    xmean = xsum/nv
    ymean = ysum/nv
    zmean = zsum/nv
    v[:,0]-= xmean 
    v[:,1]-= ymean 
    v[:,2]-= zmean 
    
# compute surface area by summing area of all facets   
# divide by 4pi which is the surface area of a sphere with radius 1
def surface_area(mesh):
    #f = mesh.faces
    S_i = mesh.get_face_attribute('face_area') 
    area =np.sum(S_i)
    return area/(4*np.pi)


# print number of faces
def nf_mesh(mesh):   
    f = mesh.faces
    print('number of faces ',len(f))

In [4]:

# meshplot with a bounding square
def plt_mesh_square(vertices,faces,xmax):
    m = np.array([-xmax,-xmax,-xmax])
    ma = np.abs(m)

    # Corners of the bounding box
    v_box = np.array([[-xmax, -xmax, 0], [-xmax, xmax, 0], [xmax, xmax,0] , [xmax, -xmax, 0]])

    # Edges of the bounding box
    f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0]], dtype=int)

    p = meshplot.plot(vertices, faces, return_plot=True)  # plot body

    p.add_edges(v_box, f_box, shading={"line_color": "red"});
    #p.add_points(v_box, shading={"point_color": "green"})
    return p


# perform a rotation on a vertex list and return a new set of rotated vertices
# rotate about axis and via angle in radians
def rotate_vertices(vertices,axis,angle):
#     v = np.copy(vertices)
#     ### if using pymesh.Quaternion
#     qs = pymesh.Quaternion.fromAxisAngle(axis, angle)
#     nv = len(v)
#     ### loop over all vertices and do the rotation
#     for i in range(nv):
#          v[i] = qs.rotate(v[i]) ### perform rotation
#     return v      
    ## if using  np.quaternion
    v = np.copy(vertices)
    qs = quaternion.from_rotation_vector(rot=axis*angle)       
    vp=quaternion.rotate_vectors(qs, v) #####, axis=-1)  # rotate vectors 
    return vp


# compute cross product C=AxB using components
def cross_prod_xyz(Ax,Ay,Az,Bx,By,Bz):
    Cx = Ay*Bz - Az*By
    Cy = Az*Bx - Ax*Bz
    Cz = Ax*By - Ay*Bx
    return Cx,Cy,Cz

# compute face_areas and face normals from a face list and a vertex list
def face_areas(vertices,faces):
    nf = len(faces)
    S_i = np.zeros(nf)
    f_normals = np.zeros((nf,3))
    for iface in range(nf):
        iv1 = faces[iface,0]  # indexes of the 3 vertices
        iv2 = faces[iface,1]
        iv3 = faces[iface,2]
        v1 = vertices[iv1]  # the 3 vertices
        v2 = vertices[iv2]
        v3 = vertices[iv3]
        e1 = v2 - v1   # edge vectors
        e2 = v3 - v2
        ax,ay,az=cross_prod_xyz(e1[0],e1[1],e1[2],e2[0],e2[1],e2[2])  # cross product
        area = np.sqrt(ax**2 + ay**2 + az**2)  # area of parallelogram
        S_i[iface] = 0.5*area #  1/2 area of parallelogram spanned by e1, e2 is area of triangle
        f_normals[iface,0] = ax/area  #normalize vector cross product 
        f_normals[iface,1] = ay/area
        f_normals[iface,2] = az/area
        # signs checked by plotting normal components in color
    return S_i,f_normals




In [5]:
# compute the volume of the tetrahedron formed from face with index iface
# and the origin
def vol_i(mesh,iface):
    f = mesh.faces
    v = mesh.vertices
    iv1 = f[iface,0]  # indexes of the 3 vertices
    iv2 = f[iface,1]
    iv3 = f[iface,2]
    #print(iv1,iv2,iv3)
    v1 = v[iv1]  # the 3 vertices
    v2 = v[iv2]
    v3 = v[iv3]
    #print(v1,v2,v3)
    mat = np.array([v1,v2,v3])  
    # the volume is equal to 1/6 determinant of the matrix formed with the three vertices
    # https://en.wikipedia.org/wiki/Tetrahedron
    #print(mat)
    vol = np.linalg.det(mat)/6.0  # compute determinant
    return vol

# compute the volume of the mesh by looping over all tetrahedrons formed from the faces
# we assume that the body is convex
def volume_mesh(mesh):
    f = mesh.faces
    nf = len(f)
    vol = 0.0
    for iface in range(nf):
        vol += vol_i(mesh,iface)
    return vol
        
# if vol equ radius is 1  the volume should be equal to 4*np.pi/3 which is 4.1888 
    

# correct all the radii so that the volume becomes that of a sphere with radius 1
# return a new mesh
def cor_volume(mesh):
    vol = volume_mesh(mesh)
    print('Volume {:.4f}'.format(vol))
    rad = pow(vol*3/(4*np.pi),1.0/3.0)
    print('radius of vol equ sphere {:.4f}'.format(rad))
    f = mesh.faces
    v = np.copy(mesh.vertices)
    v /= rad
    newmesh = pymesh.form_mesh(v, f)
    newmesh.add_attribute("face_area")
    newmesh.add_attribute("face_normal")
    newmesh.add_attribute("face_centroid")
    vol = volume_mesh(newmesh)
    print('new Volume {:.3f}'.format(vol))
    return newmesh
    

In [6]:
# compute the radiation force instantaneously on a triangular mesh for each facit
# arguments:  
#     S_i  vector of face areas
#     f_normal vector of face normals
#     s_hat is a 3 length np.array (a unit vector) pointing to the Sun
# return the vector F_i for each facet
# returns:  F_i_x is the x component of F_i and is a vector that has the length of the number of faces
# Force is zero if facets are not on the day side
def F_i(S_i,f_normal,s_hat):
    s_len = np.sqrt(s_hat[0]**2 + s_hat[1]**2 + s_hat[2]**2)  # in case s_hat was not normalized
    
    # normal components 
    nx = np.squeeze(f_normal[:,0])    # a vector, of length number of facets
    ny = np.squeeze(f_normal[:,1]) 
    nz = np.squeeze(f_normal[:,2]) 
    # dot product of n_i and s_hat
    n_dot_s = (nx*s_hat[0] + ny*s_hat[1] + nz*s_hat[2])/s_len  # a vector
    F_i_x = -S_i*n_dot_s*nx #  a vector, length number of facets
    F_i_y = -S_i*n_dot_s*ny
    F_i_z = -S_i*n_dot_s*nz
    ii = (n_dot_s <0)  # the night sides 
    F_i_x[ii] = 0  # get rid of night sides
    F_i_y[ii] = 0
    F_i_z[ii] = 0
    return F_i_x,F_i_y,F_i_z   # these are each vectors for each face

# compute radiation forces F_i for each face, but averaging over all positions of the Sun
# a circular orbit for the asteroid is assumed
# arguments: 
#    f_areas a vector of face areas
#    f_normals a vector of face normals
#    nphi_Sun is the number of solar angles, evenly spaced in 2pi so we are assuming circular orbit
#    incl is solar orbit inclination in radians
# returns: F_i_x average and other 2 components of forces for each facet
def F_i_sun_ave(f_areas,f_normals,nphi_Sun,incl):
    dphi = 2*np.pi/nphi_Sun
    # compute the first set of forces so we have vectors the right length
    phi = 0.0
    s_hat = np.array([np.cos(phi)*np.cos(incl),np.sin(phi)*np.cos(incl),np.sin(incl)])
    # compute the radiation force instantaneously on the triangular mesh for sun at s_hat
    F_i_x_sum,F_i_y_sum,F_i_z_sum = F_i(f_areas,f_normals,s_hat)
    # now compute the forces for the rest of the solar angles
    for i in range(1,nphi_Sun): # do the rest of the angles
        phi = i*dphi
        s_hat = np.array([np.cos(phi)*np.cos(incl),np.sin(phi)*np.cos(incl),np.sin(incl)])
        # compute the radiation force instantaneously on the triangular mesh for sun at s_hat
        F_i_x,F_i_y,F_i_z = F_i(f_areas,f_normals,s_hat)  
        # These are vectors of length number of facets
        F_i_x_sum += F_i_x  # sum up forces
        F_i_y_sum += F_i_y
        F_i_z_sum += F_i_z
    F_i_x_ave = F_i_x_sum/nphi_Sun  # average
    F_i_y_ave = F_i_y_sum/nphi_Sun
    F_i_z_ave = F_i_z_sum/nphi_Sun
    return F_i_x_ave,F_i_y_ave,F_i_z_ave  # these are vectors for each face




# compute total BYORP averaging over nphi_Sun solar positions
# for a single binary vector a_bin and body position described with mesh
# arguments:
#    f_areas a vector of face areas
#    f_normals a vector of face normals
#    incl is solar orbit inclination in radians
#    nphi_Sun is the number of solar angles
#    a_bin is binary direction unit vector
# returns: torque components
def tau_Bs(f_areas,f_normals,nphi_Sun,incl,a_bin):
    # compute F_i for each face, but averaging over all positions of the Sun
    F_i_x_ave, F_i_y_ave,F_i_z_ave = F_i_sun_ave(f_areas,f_normals,nphi_Sun,incl) 
    # these are vectors length number of faces
    # forces from day lit faces
    F_x = np.sum(F_i_x_ave)  #sum up the force
    F_y = np.sum(F_i_y_ave)
    F_z = np.sum(F_i_z_ave)
    a_x = a_bin[0]  # binary direction
    a_y = a_bin[1]
    a_z = a_bin[2]
    tau_x,tau_y,tau_z = cross_prod_xyz(a_x,a_y,a_z,F_x,F_y,F_z) # cross product
    return tau_x,tau_y,tau_z  # these are numbers that give the torque components
        

In [7]:
# first rotate vertices in the mesh about the z axis by angle phi in radians
# then tilt over the body by obliquity which is an angle in radians
# arguments:
#    vertices, triangular mesh surface vertices for body
#    obliquity, angle in radius to tilt body z axis over
#    phi, angle in radians to spin/rotate body about its z axis
#    phi_prec,  angle in randias that tilt is done, it's a precession angle
#      sets rotation axis for tilt, this axis is in the xy plane
# returns: 
#     new_mesh: the tilted/rotated mesh
#     zrot:  the new z-body spin axis
def tilt_obliq(vertices,obliquity,phi,phi_prec):
    #f = mesh.faces
    v = np.copy(vertices)
    #nv = len(v)
    axist = np.array([np.cos(phi_prec),np.sin(phi_prec),0])  # precession angle is phi_prec
    qt = quaternion.from_rotation_vector(rot=axist*obliquity)     # tilt    
    vp=quaternion.rotate_vectors(qt, v) #####, axis=-1)  # rotate vectors, tilt over
    zaxis = np.array([0,0,1])
    zrot = quaternion.rotate_vectors(qt,zaxis) # body principal axis will become zrot
    qs = quaternion.from_rotation_vector(rot=zrot*phi)    # spin rotation
    vpp = quaternion.rotate_vectors(qs, vp)   # rotate vectors spin
    return vpp,zrot
    

# tilt,spin a body and compute binary direction, assuming tidally locked
# arguments:
#   vertices of a mesh body (in principal axis coordinate system)
#   nphi is the number of angles that could be done with indexing by iphi
#   obliquity:  w.r.t to binary orbit angular momentum direction
#   iphi:  number of rotations by dphi where dphi = 2pi/nphi
#      this is principal axis rotation about z axis
#   phi0: an offset for phi applied to body but not binary axis, similar to a libration angle, but not quite
#      as is not projected onto orbital plane, is an offset in rotation angle
#   phi_prec: a precession angle for tilting
# returns: 
#   tvertices, a body of vertices rotated after iphi rotations by dphi and tilted by obliquity
#   a_bin, binary direction assuming same rotation rate, tidal lock
#   l_bin:  binary orbit angular momentum orbital axis
#   zrot:  spin axis direction 
def tilt_and_bin(vertices,obliquity,nphi,iphi,phi0,phi_prec):
    dphi = 2*np.pi/nphi
    phi = iphi*dphi 
    tvertices,zrot = tilt_obliq(vertices,obliquity,phi + phi0,phi_prec)  # tilt and spin body
    a_bin = np.array([np.cos(phi),np.sin(phi),0.0])   # direction to binary
    l_bin = np.array([0,0,1.0])  # angular momentum axis of binary orbit
    return tvertices,a_bin,l_bin,zrot

In [8]:

# compute the BYORP torque, for a tidally locked binary
# arguments:
#   body:  triangular surface mesh (in principal axis coordinate system)
#   nphi is the number of body angles we will use (spin)
#   obliquity is body tilt w.r.t to binary orbit
#   incl is solar orbit inclination 
#   nphi_Sun is the number of solar angles used
#   phi0 an offset for body spin angle that is not applied to binary direction
#   phi_prec  z-axis precession angle, relevant for obliquity 
# returns:
#   3 torque components
#   torque dot l_bin so can compute binary orbit drift rate
def compute_BY(vertices,faces,obliquity,nphi,nphi_Sun,incl,phi0,phi_prec):
    tau_BY_x = 0.0
    tau_BY_y = 0.0
    tau_BY_z = 0.0
    for iphi in range(nphi):  # body positions
        # rotate the body and tilt it over, and find binary direction
        tvertices,a_bin,l_bin,zrot = tilt_and_bin(vertices,obliquity,nphi,iphi,phi0,phi_prec)
        # rotated body is in tvertices
        # now recompute face areas and surface normal directions
        f_areas, f_normals = face_areas(tvertices,faces) # vector of facet areas and face normal vectors
        # of the rotated body, faces don't change
        # a_bin is binary direction
        # compute torques 
        tau_x,tau_y,tau_z =tau_Bs(f_areas,f_normals,nphi_Sun,incl,a_bin)  # incl and a_bin here
        tau_BY_x += tau_x
        tau_BY_y += tau_y
        tau_BY_z += tau_z
        
    tau_BY_x /= nphi  # average
    tau_BY_y /= nphi
    tau_BY_z /= nphi
    
    # recompute l_bin
    tvertices,a_bin,l_bin,zrot = tilt_and_bin(vertices,obliquity,nphi,0,phi0,phi_prec)
         
    # compute component that affects binary orbit angular momentum
    # this is tau dot l_bin
    tau_l = tau_BY_x*l_bin[0] + tau_BY_y*l_bin[1] + tau_BY_z*l_bin[2] 
    return tau_BY_x,tau_BY_y,tau_BY_z, tau_l 

In [9]:
print(4*np.pi/3)

4.1887902047863905


In [10]:

# compute the BYORP torque on body as a function of obliquity
# for a given inclination and precession angle, and rotation offset angle phi0
# returns obliquity and torque arrays
def obliq_BY_fig2(vertices,faces,incl,phi_prec,phi0):
    #phi0=0
    nphi_Sun=36  # number of solar positions
    nphi = 36    # number of spin positions
    nobliq = 60   # number of obliquities
    dobliq = np.pi/nobliq
    tau_l_arr = np.zeros(nobliq)  # to store torques
    o_arr = np.zeros(nobliq)
    for i in range(nobliq):
        obliquity=i*dobliq
        tau_BY_x,tau_BY_y,tau_BY_z, tau_l =compute_BY(vertices,faces,obliquity,nphi,nphi_Sun,incl,phi0,phi_prec)
        o_arr[i] = obliquity*180/np.pi
        tau_l_arr[i] = tau_l
    return o_arr,tau_l_arr

In [11]:
# create a sphere of radius 1
center = np.array([0,0,0])
sphere = pymesh.generate_icosphere(1., center, refinement_order=2)
sphere.add_attribute("face_area")
sphere.add_attribute("face_normal")
sphere.add_attribute("face_centroid")
print(volume_mesh(sphere))
nf_mesh(sphere)

# create a perturbed ellipsoid using the above sphere
devrand = 0.05  # perturbation size  # fiducial model
aratio1 = 0.5   # axis ratios c/a
aratio2 = 0.7   # b/a
random.seed(2)  # fix the sequence
psphere1 = sphere_perturb(sphere,devrand,1,1)  #perturbed sphere 
body1 = body_stretch(psphere1,aratio1,aratio2)  # stretch 
#print(volume_mesh(body1))  #check volume


4.047005367354512
number of faces  320


In [12]:
devrand = 0.025  # perturbation size larger
random.seed(2)  # same perturbations
psphere2 = sphere_perturb(sphere,devrand,1,1)  #less perturbed sphere 
aratio1 = 0.5   # same axis ratios
aratio2 = 0.7
body2 = body_stretch(psphere2,aratio1,aratio2)

devrand = 0.05  # perturbation size same as fiducial
random.seed(21)  # different perturbations
psphere3 = sphere_perturb(sphere,devrand,1,1)  #fid perturbed sphere 
aratio1 = 0.5   # same axis ratios
aratio2 = 0.7
body3 = body_stretch(psphere3,aratio1,aratio2)


devrand = 0.05  # perturbation size same as fiducial
random.seed(25)  # different perturbations
psphere4 = sphere_perturb(sphere,devrand,1,1)  #fid perturbed sphere 
aratio1 = 0.5   # same axis ratios
aratio2 = 0.7
body4 = body_stretch(psphere4,aratio1,aratio2)


devrand = 0.05  # perturbation size same as fiducial
random.seed(29)  # different perturbations
psphere5 = sphere_perturb(sphere,devrand,1,1)  #fid perturbed sphere 
aratio1 = 0.5   # same axis ratios
aratio2 = 0.7
body5 = body_stretch(psphere5,aratio1,aratio2)

devrand = 0.05  # perturbation size same as fiducial
random.seed(31)  # different perturbations
psphere6 = sphere_perturb(sphere,devrand,1,1)  #fid perturbed sphere 
aratio1 = 0.5   # same axis ratios
aratio2 = 0.7
body6 = body_stretch(psphere6,aratio1,aratio2)



In [13]:
S_i, f_normals = face_areas(body1.vertices,body1.faces)
#d = meshplot.subplot(body1.vertices, body1.faces, c=v[:, 1], s=[2, 2, 2])
d=meshplot.subplot(body1.vertices, body1.faces, c=S_i, s=[2, 2, 2])
meshplot.subplot(body1.vertices, body1.faces, c=f_normals[:,2], s=[2, 2, 2])

In [14]:
xmax = 1.5
p = plt_mesh_square(body1.vertices,body1.faces,xmax)
# p.save('junk.html')  # works but no way to snap or get orientation 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.026061…

In [15]:
xrot = np.array([1,0,0])
vrot = rotate_vertices(body1.vertices,xrot,np.pi/2)
p = plt_mesh_square(vrot,body1.faces,xmax)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.026061…

In [16]:
xmax = 1.5
p = plt_mesh_square(body2.vertices,body1.faces,xmax)
# p.save('junk.html')  # works but no way to snap or get orientation 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.013030…

In [17]:
xrot = np.array([1,0,0])
vrot = rotate_vertices(body2.vertices,xrot,np.pi/2)
p = plt_mesh_square(vrot,body2.faces,xmax)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.013030…

In [18]:

aratio1 = 0.5   # new axis ratios
aratio2 = 0.8
body1a = body_stretch(psphere1,aratio1,aratio2)

aratio1 = 0.6   # new axis ratios
aratio2 = 0.7
body1b = body_stretch(psphere1,aratio1,aratio2)


aratio1 = 0.4   # new axis ratios
aratio2 = 0.9
body1c = body_stretch(psphere1,aratio1,aratio2)



In [19]:
# check total surface area
print(surface_area(sphere))
print(surface_area(psphere1))
print(surface_area(psphere2))
print(surface_area(psphere3))
print(surface_area(body1))
print(surface_area(body2))
print(surface_area(body3))
print(surface_area(body1a))
print(surface_area(body1b))

# subtract 1 and you have approximately the d_s used by Steinberg+10
# many of their d_s are  lower (see their figure 3)

0.9811156432317085
1.003830499198687
0.9883108907346502
0.9992616460925486
1.0664364654025773
1.050536574113009
1.0643474288238175
1.072882983618328
1.0380017699710702


In [21]:
# compute BYORPs as a function of obliquity
incl = 0; phi_prec=0; phi0=0
o_arr1 ,tau_l_arr1 = obliq_BY_fig2(body1.vertices,body1.faces,incl,phi_prec,phi0)
o_arr_s,tau_l_arr_s = obliq_BY_fig2(sphere.vertices,sphere.faces,incl,phi_prec,phi0)
o_arr2,tau_l_arr2 = obliq_BY_fig2(body2.vertices,body2.faces,incl,phi_prec,phi0)
o_arr3,tau_l_arr3 = obliq_BY_fig2(body3.vertices,body3.faces,incl,phi_prec,phi0)
o_arr4,tau_l_arr4 = obliq_BY_fig2(body4.vertices,body4.faces,incl,phi_prec,phi0)
o_arr5,tau_l_arr5 = obliq_BY_fig2(body5.vertices,body5.faces,incl,phi_prec,phi0)
o_arr6,tau_l_arr6 = obliq_BY_fig2(body6.vertices,body6.faces,incl,phi_prec,phi0)

In [ ]:
v_cmshift = np.copy(body6.vertices)
v_cmshift[:,2] -= 0.05 # shift the center of mass, no affect! 
#o_arr7,tau_l_arr7 = obliq_BY_fig2(v_cmshift,body6.faces,incl,phi_prec,phi0)

In [ ]:
v_cmshift = np.copy(body6.vertices)
v_cmshift[:,2] -= 0.05 # shift the center of mass, no affect! 
#o_arr7,tau_l_arr7 = obliq_BY_fig2(v_cmshift,body6.faces,incl,phi_prec,phi0)
fig,ax = plt.subplots(1,1,figsize=(6,3),dpi=300)
plt.subplots_adjust(bottom=0.19,top=0.98,left=0.2, right=0.98) 
ax.plot(o_arr_s,tau_l_arr_s,'go:',alpha=0.5,label='icosphere')
ax.plot(o_arr1,tau_l_arr1,'bd:',alpha=0.5,label=r'$\Delta=0.05$')
ax.plot(o_arr3,-tau_l_arr3,'mP:',alpha=0.5,label=r'$\Delta=0.05$')
ax.plot(o_arr2,tau_l_arr2,'rs:',alpha=0.5,label=r'$\Delta=0.025$')
ax.plot(o_arr4,tau_l_arr4,'d:',alpha=0.5,label=r'$\Delta=0.05$',color='gray')
ax.plot(o_arr5,tau_l_arr5,'H:',alpha=0.5,label=r'$\Delta=0.05$',color='brown')
ax.plot(o_arr6,-tau_l_arr6,'h:',alpha=0.6,label=r'$\Delta=0.05$',color='orange')
#ax.plot(o_arr7,-tau_l_arr7,'o:',alpha=0.6,label=r'$\Delta=0.05$',color='black')
ax.set_xlabel('obliquity (deg)',fontsize=16)
#ax.set_ylabel(r'${ \tau}_{BY} \cdot \hat{l}$',fontsize=16)
ax.set_ylabel(r'$g_{BY}$',fontsize=16)
ax.legend(borderpad=0.1,labelspacing=0.1,handletextpad=0.1,loc='lower right',ncol=2)
ax.text(130,0.010,'c/a=0.5 b/a=0.7')
plt.savefig('tau_BY_obl.png')

### This is going to zero at high obliquity because spin is counter to rotation of binary

In [ ]:
o_arr1a,tau_l_arr1a = obliq_BY_fig2(body1a.vertices,body1a.faces,incl,phi_prec,phi0)
o_arr1b,tau_l_arr1b = obliq_BY_fig2(body1b.vertices,body1b.faces,incl,phi_prec,phi0)
o_arr1c,tau_l_arr1c = obliq_BY_fig2(body1c.vertices,body1c.faces,incl,phi_prec,phi0)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,3),dpi=300)
plt.subplots_adjust(bottom=0.19,top=0.98,left=0.2,right=0.98) 

ax.plot(o_arr_s,tau_l_arr_s,'go:',alpha=0.5,label='icosphere')
ax.plot(o_arr1 ,tau_l_arr1, 'rs:',alpha=0.5,label=r'$c/a=0.5, b/a=0.7$')
ax.plot(o_arr1a,tau_l_arr1a,'bd:',alpha=0.5,label=r'$c/a=0.5, b/a=0.8$')
ax.plot(o_arr1b,tau_l_arr1b,'cv:',alpha=0.5,label=r'$c/a=0.6, b/a=0.7$')
ax.plot(o_arr1c,tau_l_arr1c,'H:',alpha=0.5,label=r'$c/a=0.4, b/a=0.9$', color='brown')

ax.set_xlabel('obliquity (deg)',fontsize=16)
#ax.set_ylabel(r'${ \tau}_{BY} \cdot \hat{l}$',fontsize=16)
ax.set_ylabel(r'$g_{BY}$',fontsize=16)
ax.legend(borderpad=0.1,labelspacing=0.1,handletextpad=0.1)
ax.text(150,-0.0018, r'$\Delta=0.05$')
plt.savefig('tau_BY_obl2.png')

In [ ]:
print(o_arr1c[0:10])
print(tau_l_arr1c[0:10])

In [ ]:
#https://www.naic.edu/~smarshal/1999kw4.html
#Squannit is secondary of Moshup which was 1999KW4

squannit = pymesh.load_mesh("kw4b.obj")
nf_mesh(squannit)
# we need to normalize it so that its volume is 4/3 pi
# to compute the volume of a tetrahdon that is made from a face + a vertex at the origin 
# we need to compute the determanent of a 3x3 matrix that consists of the 3 vertices in the face.
# we then sum over all faces in the mesh to get the total volume
# alternatively we use the generalized voxel thing in pymesh which is 4 vertices.
# to do this we add a vertex at the center and then we need to make the same number
# of voxels as faces using the vertex at the origin.
# and then we sum over the voxel_volume attributes of all the voxels.

In [ ]:
xmax = 1.5
p = plt_mesh_square(squannit.vertices,squannit.faces,xmax)

In [ ]:
vol = volume_mesh(squannit)
print(vol)
R_squannit = pow(vol*3/(4.0*np.pi),0.3333333)
print(R_squannit)  # I don't know what units this is in. maybe km
# the object is supposed to have Beta: 0.571 x 0. 463 x 0.349 km (6% uncertainty)

In [ ]:
# rescale so it has vol equiv sphere radius of 1
new_squannit = cor_volume(squannit)
p = plt_mesh_square(new_squannit.vertices,new_squannit.faces,xmax)

In [ ]:
xrot = np.array([1,0,0])
vrot = rotate_vertices(new_squannit.vertices,xrot,np.pi/2)
p = plt_mesh_square(vrot,new_squannit.faces,xmax)

In [ ]:
# reduce the number of faces to something reasonable
short_squannit1, info = pymesh.collapse_short_edges(new_squannit, 0.219)  # if bigger then  fewer faces
nf_mesh(short_squannit1)
meshplot.plot(short_squannit1.vertices, short_squannit1.faces)
short_squannit2, info = pymesh.collapse_short_edges(new_squannit, 0.17)  # if bigger then  fewer faces
nf_mesh(short_squannit2)
meshplot.plot(short_squannit2.vertices, short_squannit2.faces)

xrot = np.array([1,0,0])
vrot = rotate_vertices(short_squannit2.vertices,xrot,np.pi/2)  # perturbed version of squannit

In [ ]:
p = plt_mesh_square(short_squannit2.vertices,short_squannit2.faces,xmax)

In [ ]:

p = plt_mesh_square(vrot,short_squannit2.faces,xmax)

In [ ]:
devrand=0.05
short_squannit2_p  = sphere_perturb(short_squannit2,devrand,1,1)
p = plt_mesh_square(short_squannit2_p.vertices,short_squannit2_p.faces,xmax)

In [ ]:
xrot = np.array([1,0,0])
vrot = rotate_vertices(short_squannit2_p.vertices,xrot,np.pi/2)
p = plt_mesh_square(vrot,short_squannit2_p.faces,xmax)

In [ ]:
# compute BYORPs as a function of obliquity
incl = 0; phi_prec=0; phi0=0
o_arr1,stau_l_arr1 = obliq_BY_fig2(short_squannit1.vertices,short_squannit1.faces,incl,phi_prec,phi0)

In [ ]:
# compute BYORPs as a function of obliquity
incl = 0; phi_prec=0; phi0=0
o_arr2,stau_l_arr2 = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)

In [ ]:
# compute BYORPs as a function of obliquity, perturbed
incl = 0; phi_prec=0; phi0=0
o_arr2_p,stau_l_arr2_p = obliq_BY_fig2(short_squannit2_p.vertices,short_squannit2_p.faces,incl,phi_prec,phi0)

In [ ]:
# compute BYORPs as a function of obliquity, vary phi_prec
incl = 0; phi_prec=10*np.pi/180; phi0=0
o_arr3a,stau_l_arr3a = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)
incl = 0; phi_prec=-10*np.pi/180; phi0=0
o_arr3b,stau_l_arr3b = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)

In [ ]:
# compute BYORPs as a function of obliquity, vary incl
incl = 20*np.pi/180; phi_prec=0; phi0=0
o_arr4a,stau_l_arr4a = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)
incl = 160*np.pi/180; phi_prec=0; phi0=0
o_arr4b,stau_l_arr4b = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)

In [ ]:
# compute BYORPs as a function of obliquity, vary lib
incl = 0; phi_prec=0; phi0=20*np.pi/180
o_arr5a,stau_l_arr5a = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)
incl = 0; phi_prec=0; phi0=-20*np.pi/180
o_arr5b,stau_l_arr5b = obliq_BY_fig2(short_squannit2.vertices,short_squannit2.faces,incl,phi_prec,phi0)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,3),dpi=150)
plt.subplots_adjust(bottom=0.19, top = 0.98,left=0.2,right=0.98)
#ax.plot(o_arr2,stau_l_arr2,'go-',label='sphere')
ax.plot(o_arr1,-stau_l_arr1,'rD-',label=r'S302',alpha=0.5)
ax.plot(o_arr2,-stau_l_arr2,'bv-',label=r'S534',alpha=0.5)
ax.plot(o_arr2_p,-stau_l_arr2_p,'bP-',label=r'S534 $\Delta=0.05$',alpha=0.5)
#ax.plot(o_arr3a,stau_l_arr3a,'gP-',label=r'S534, $\phi_p=10^\circ$',alpha=0.5)
#ax.plot(o_arr3b,stau_l_arr3b,'gP-',label=r'S534, $\phi_p=-10^\circ$',alpha=0.5)
#ax.plot(o_arr4a,-stau_l_arr4a,'cH-',label=r'S534, $i_h=20^\circ$',alpha=0.5)
ax.plot(o_arr4b,-stau_l_arr4b,'cH-',label=r'S534, $i_h=160^\circ$',alpha=0.5)
ax.plot(o_arr5a,-stau_l_arr5a,'ms-',label=r'S534, $\phi_l=20^\circ$',alpha=0.5)
ax.plot(o_arr5b,-stau_l_arr5b,'ms-',label=r'S534, $\phi_l=-20^\circ$',alpha=0.5)
ax.set_xlabel('obliquity (deg)',fontsize=16)
#ax.set_ylabel(r'${ \tau}_{BY} \cdot \hat{l}$',fontsize=16)
ax.set_ylabel(r'$g_{BY}$',fontsize=16)

ax.legend(loc='upper right',borderpad=0.1,labelspacing=0.1,ncol=2)
plt.savefig('squannit.png')

In [ ]:
#Psudocode for what AJ has to do 
"""
#TODO: ### **NPA rotation implementation into BYORP calculation**
- [ ] Random rotation calculation
    - normal distribution
    - run primary loop for 1 -> ~90degrees
- [ ] Tilt and bin
    - Tilt the secondary to face the primary 
    - Tilt around random number for NPA rotation
- [ ] Calcualte BYORP effect on various meshes
    - Squannit
    - 3 other mesh (look in notebook)
    
#TDOD: ### Put in exponential timescales into paper
"""

import random as r
import numpy as np

fun = np.array([])

def getRandDeg(maxAng):
#     print("Max ", maxAng,". Min ", -maxAng,".")
    return maxAng*r.uniform(-1,1)



# first rotate vertices in the mesh about the z axis by angle phi in radians
# then tilt over the body by obliquity which is an angle in radians
# arguments:
#    vertices, triangular mesh surface vertices for body
#    obliquity, angle in radius to tilt body z axis over
#    phi, angle in radians to spin/rotate body about its z axis
#    phi_prec,  angle in randias that tilt is done, it's a precession angle
#      sets rotation axis for tilt, this axis is in the xy plane
# returns: 
#     new_mesh: the tilted/rotated mesh
#     zrot:  the new z-body spin axis
def tilt_NPA(vertices,NPAMax,phi,phi_prec):
    v = np.copy(vertices)
    axist = np.array([np.cos(phi_prec),np.sin(phi_prec),0])  # precession angle is phi_prec
    
    zaxis = np.array([0,0,1])
    qspin = quaternion.from_rotation_vector(rot=zaxis*phi)    # spin rotation
    vS = quaternion.rotate_vectors(qspin,v)
    xrot = quaternion.rotate_vectors(qspin,np.array([1,0,0]))
    
    currRot = getRandDeg(NPAMax)
#     print("i")
    global fun 
    fun = np.append(fun,currRot)
    qrot = quaternion.from_rotation_vector(rot=xrot*currRot)
    vSRot = quaternion.rotate_vectors(qrot,vS)
    
    
#     qt = quaternion.from_rotation_vector(rot=axist*obliquity)     # tilt    
#     vp=quaternion.rotate_vectors(qt, v) #####, axis=-1)  # rotate vectors, tilt over
    
    
#     zrot = quaternion.rotate_vectors(qt,zaxis) # body principal axis will become zrot
#     zaxis = np.array([0,0,1])
#     qs = quaternion.from_rotation_vector(rot=zrot*phi)    # spin rotation
#     vpp = quaternion.rotate_vectors(qs, vp)   # rotate vectors spin
    return vSRot,zaxis
    

# tilt,spin a body and compute binary direction, assuming tidally locked
# arguments:
#   vertices of a mesh body (in principal axis coordinate system)
#   nphi is the number of angles that could be done with indexing by iphi
#   obliquity:  w.r.t to binary orbit angular momentum direction
#   iphi:  number of rotations by dphi where dphi = 2pi/nphi
#      this is principal axis rotation about z axis
#   phi0: an offset for phi applied to body but not binary axis, similar to a libration angle, but not quite
#      as is not projected onto orbital plane, is an offset in rotation angle
#   phi_prec: a precession angle for tilting
# returns: 
#   tvertices, a body of vertices rotated after iphi rotations by dphi and tilted by obliquity
#   a_bin, binary direction assuming same rotation rate, tidal lock
#   l_bin:  binary orbit angular momentum orbital axis
#   zrot:  spin axis direction 
def tilt_and_bin(vertices,NPAMax,nphi,iphi,phi0,phi_prec):
    dphi = 2*np.pi/nphi
    phi = iphi*dphi 
    tvertices,zrot = tilt_NPA(vertices,NPAMax,phi + phi0,phi_prec)  # tilt and spin body
    a_bin = np.array([np.cos(phi),np.sin(phi),0.0])   # direction to binary
    l_bin = np.array([0,0,1.0])  # angular momentum axis of binary orbit
    return tvertices,a_bin,l_bin,zrot


# compute the BYORP torque on body as a function of obliquity
# for a given inclination and precession angle, and rotation offset angle phi0
# returns obliquity and torque arrays
def NPA_BY_fig2(vertices,faces,incl,phi_prec,phi0):
    global fun
    fun = np.array([])
    #phi0=0
    nphi_Sun= 128  # number of solar positions
    nphi = 128    # number of spin positions
    nNPA = 180   # number of obliquities
    dNPA = np.pi/nNPA
    tau_l_arr = np.zeros(nNPA)  # to store torques
    r_arr = np.zeros(nNPA)
    for i in range(nNPA):
        NPAMax=i*dNPA
#         print("Funny NPAMax,", NPAMax*180/np.pi)
        tau_BY_x,tau_BY_y,tau_BY_z, tau_l =compute_BY(vertices,faces,NPAMax,nphi,nphi_Sun,incl,phi0,phi_prec)
        r_arr[i] = NPAMax*180/np.pi
        tau_l_arr[i] = tau_l
    fun = fun.reshape(nNPA,nphi+1)
#     print(fun)
    return r_arr,tau_l_arr


In [ ]:
# compute BYORPs as a function of obliquity
incl = 0; phi_prec=0; phi0=0
o_arr1 ,tau_l_arr1 = NPA_BY_fig2(body1.vertices,body1.faces,incl,phi_prec,phi0)
print("finish1")
o_arr_s,tau_l_arr_s = NPA_BY_fig2(sphere.vertices,sphere.faces,incl,phi_prec,phi0)
print("finishs")
o_arr2,tau_l_arr2 = NPA_BY_fig2(body2.vertices,body2.faces,incl,phi_prec,phi0)
print("finish2")
o_arr3,tau_l_arr3 = NPA_BY_fig2(body3.vertices,body3.faces,incl,phi_prec,phi0)
print("finish3")
o_arr4,tau_l_arr4 = NPA_BY_fig2(body4.vertices,body4.faces,incl,phi_prec,phi0)
print("finish4")
o_arr5,tau_l_arr5 = NPA_BY_fig2(body5.vertices,body5.faces,incl,phi_prec,phi0)
print("finish5")
o_arr6,tau_l_arr6 = NPA_BY_fig2(body6.vertices,body6.faces,incl,phi_prec,phi0)
print("finish6")
v_cmshift = np.copy(body6.vertices)
v_cmshift[:,2] -= 0.05 # shift the center of mass, no affect! 
#o_arr7,tau_l_arr7 = obliq_BY_fig2(v_cmshift,body6.faces,incl,phi_prec,phi0)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,6),dpi=300)
plt.subplots_adjust(bottom=0.19,top=0.98,left=0.2, right=0.98) 
ax.plot(o_arr_s,tau_l_arr_s,'go:',alpha=0.5,label='icosphere')
ax.plot(o_arr1,tau_l_arr1,'bd:',alpha=0.5,label=r'$\Delta=0.05$')
ax.plot(o_arr3,-tau_l_arr3,'mP:',alpha=0.5,label=r'$\Delta=0.05$')
ax.plot(o_arr2,tau_l_arr2,'rs:',alpha=0.5,label=r'$\Delta=0.025$')
ax.plot(o_arr4,tau_l_arr4,'d:',alpha=0.5,label=r'$\Delta=0.05$',color='gray')
ax.plot(o_arr5,tau_l_arr5,'H:',alpha=0.5,label=r'$\Delta=0.05$',color='brown')
ax.plot(o_arr6,-tau_l_arr6,'h:',alpha=0.6,label=r'$\Delta=0.05$',color='orange')
#ax.plot(o_arr7,-tau_l_arr7,'o:',alpha=0.6,label=r'$\Delta=0.05$',color='black')
ax.set_xlabel(r'$NPA_{max}$ (deg)',fontsize=16)
#ax.set_ylabel(r'${ \tau}_{BY} \cdot \hat{l}$',fontsize=16)
ax.set_ylabel(r'$g_{BY}$',fontsize=16)
ax.legend(borderpad=0.1,labelspacing=0.1,handletextpad=0.1,loc='lower right',ncol=2)
ax.text(130,0.010,'c/a=0.5 b/a=0.7')
plt.savefig('tau_BY_obl.png')